# Defactoring Vossanto

In dieser Arbeit wird der Code zum Paper *"The Michael Jordan of greatness". Extracting Vossian antonomasia from two decades of The New York Times, 1987-2007* von Fischer und Jäschke aus dem Jahr 2019 untersucht. Der für diese Studie verwendete Code wurde in einem [Github Repositorium](https://github.com/weltliteratur/vossanto/tree/master/theof) veröffentlicht. Sofern nicht explizit anderweitig erwähnt, stammt sämtlicher in diesem Notebook verwendeter Code aus dem erwähnten Repositorium.
Der Code ist überwiegend in der Programmiersprache Python geschrieben. Zur Veranschaulichung von Zwischenergebnissen werden außerdem Shellbefehle verwendet. So können Textdateien mit der Skriptsprache *awk* direkt im Dateisystem ausgewertet werden. Das Notebook folgt damit der Methodik von Fischer und Jäschke zur Veranschaulichung der Daten.

## Vorbereitung

In diesem Abschnitt werden Vorbereitungen für die Analyse der Daten vorgenommen. Dazu gehört u. a. das Laden benötigter Bibliotheken. Dieser Schritt stellt eine Schnittstelle zwischen der projektspezifischen Software, welche gezielt für diese Studie geschrieben wurde, und allgemeinerer Software und Infrastruktur dar. Diese Brücke wird durch das Schlüsselwort *import* deutlich, mit dem die Funktionalität eines existierenden Programms in das hier geschriebene eingefügt wird und wiederverwendet werden kann.

Anschließend werden in diesem Auswertungsschritt benötigte Parameter wie Dateipfade und verwendete Muster regulärer Ausdrücke (Regex) konfiguriert.

### Imports

In [1]:
### DEFACTORING IMPORT

import re
import xml.etree.ElementTree as ET
import os
import codecs
from nltk.tokenize import sent_tokenize

Von den geladenen Bibliotheken kann die Mehrzahl der allgemeinen Softwareebene in [Hinsens Modell](http://blog.khinsen.net/posts/2017/01/13/sustainable-software-and-reproducible-research-dealing-with-software-collapse/) zugeordnet werden. Dazu gehören *re*, *xml*, *os* und *codecs*. All diese sind Standardbibliotheken, die von Python bereit gestellt und nicht speziell im wissenschaftlichen Kontext angewandt werden. Die Bibliothek *nltk* gehört zur disziplinspezifischen Software. Sie ist eine weit verbreitete Bibliothek im Bereich *Natural Language Processing*. Für diesen Anwendungsfall wird lediglich die Methode *sent_tokenize* importiert, die Texte in Sätze aufteilt.  
Allein die Betrachtung dieser Imports gibt bereits Hinweise auf die Art des erzeugten Programms. Ohne dass konkrete Anweisungen und Methoden für den Umgang mit Eingabewerten erkennbar sind, wird bereits deutlich, dass das Programm mit dem Dateisystem arbeitet und dabei Text verarbeitet wird, wobei das XML-Format eine Rolle spielt.

Der originale Quellcode enthält eine Reihe Bibliotheken, die nach dem Defactoring nicht mehr benötigt werden. Einige entfallen durch das Defactoring, weil der Code für eine bessere Lesbarkeit umgeschrieben wurde. Dazu gehören *argparse*, *gzip*, *tarfile* und *sys*. Diese allgemeinen Bibliotheken wurden verwendet, um verschiedene Konfigurationen und Eingabeformate verarbeiten zu können. Da die Formate in diesem Anwendungsfall durch die zur Verfügung gestellten Daten feststehen, wurde an dieser Stelle auf das Anbieten von Alternativen verzichtet.  
Außerdem wurde im Original die gesamte Bibliothek *nltk* zusätzlich zur Methode *sent_tokenize* importiert, in dieser Form jedoch nicht weiter verwendet. Im Unterschied zum Laden der gesamten Bibliothek wird mit dem Befehl *from nltk.tokenize import sent_tokenize* explizit nur die spezifische Methode *sent_tokenize* geladen. Der Vorteil davon liegt zum einen darin, dass keine unnötigen Methoden in den Speicher geladen werden. Zum anderen ist die Methode dadurch direkt über ihren Methodennamen aufrufbar, was die Lesbarkeit für Menschen vereinfacht.  
Dass im Original zusätzlich die gesamte NLTK-Bibliothek importiert wurde, weist möglicherweise darauf hin, dass zu einem früheren Zeitpunkt mit verschiedenen Methoden der Bibliothek gearbeitet wurde. Die Versionskontrolle könnte darüber Aufschluss geben. Allerdings wird schon in der ersten nachvollziehbaren Version des Codes nur die Methode *sent_tokenize* verwendet (siehe [Github-Historie](https://github.com/weltliteratur/vossanto/commit/fecfd93c478c567bce9f69eef534c4a8db73611c#diff-6c0bf7a6645955c2ca40adbff492c77c)).

### Konfiguration

In diesem Abschnitt werden Parameter eingestellt, die im weiteren Verlauf der Analyse verwendet werden. Dazu gehören die Pfade zu verwendeten Dateien (wie dem Verzeichnis der NYT-Artikeln und der Liste mit Wikidata-Einträgen zu Personen).

Die Variablen für Verzeichnisse wurden im Rahmen des Defactorings angelegt. Fischer und Jäschke arbeiten vor allem über die Kommandozeile und übergeben Werte wie Dateinamen als Argumente mit dem Aufruf eines Skripts. Dadurch ist es jedoch schwerer nachzuvollziehen, an welcher Stelle im Code die übergebene Datei konkret verarbeitet wird. Daher werden die Dateien hier direkt den betreffenden Methoden übergeben. Der besseren Lesbarkeit halber werden diese Variablen zugeordnet.

In [2]:
### DEFACTORING NAMESPACE

### Verzeichnisse und Dateien
NYT_DIRECTORY = 'nyt_corpus_1987'
WIKIDATA_HUMANS = 'wikidata_humans_prep.tsv'
BLACKLIST_FILE = 'blacklist_prep.tsv'

### zunächst leeres Set, in dem später die Blacklist gespeichert wird
blacklist = set()

### Reguläre Ausdrücke

### Muster für Vossantos
re_theof = {
    # 1: simple match
    1: re.compile("(\\bthe\\s([A-Z][a-z]+\\s+){1,3}of\\b)"),
    # 2: more than three words
    2: re.compile("(\\bthe\\s([A-Z][a-z]+\\s+)+of\\b)"),
    # 3: lower-case characters and at most five words
    3: re.compile("(\\bthe\\s([A-Za-z]+\\s+){1,5}of\\b)"),
    # 4: all unicode characters (+non-greedy)
    4: re.compile("(\\bthe\\s(\\w+\\s+){1,5}?of\\b)", re.UNICODE),
    # 5: all unicode characters and .-,'
    5: re.compile("(\\bthe\\s+([\\w.,'-]+\\s+){1,5}?of\\b)", re.UNICODE)
}

### Muster, das prüft, ob ein String "the" (umgeben von Leerzeichen) enthält
re_the = re.compile(r".*\bthe\s+(.*)$")

### Muster, das zwei aufeinander folgende Anführungszeichen identifiziert.
### (Einträge in der Wikidata-Liste haben das muster '"Q42"    "Michael Adams"')
re_quotes = re.compile("\"\"")

### Muster für Leerzeichen
re_ws = re.compile('[\n\t\r]+')

### Muster für weitere Steuerzeichen
# to remove control characters, see
# https://stackoverflow.com/questions/92438/stripping-non-printable-characters-from-a-string-in-python
control_chars = ''.join(map(chr, list(range(0,32)) + list(range(127,160))))
control_char_re = re.compile('[%s]' % re.escape(control_chars))

Zunächst werden die Dateipfade für die NYT-Artikel, die Personenliste von Wikidata und die erzeugte Blacklist in Variablen gespeichert. Für solche Werte, die sich im Laufe des Programms nicht verändern, werden Konstanten verwendet. Sie werden in Python laut [Konvention](https://www.python.org/dev/peps/pep-0008/#constants) durch Großbuchstaben gekennzeichnet. Wichtig dabei ist, dass die entsprechenden Variablen nur den Pfad zu den Dateien als String-Wert enthalten, nicht aber die Dateiinhalte selbst. Dass es sich bei einem übergebenen Wert um eine Datei handelt, muss in Python auf vorgeschriebene Art mitgeteilt werden. Dieses Vorgehen findet innerhalb später verwendeter Methoden selbst statt.

Anschließend wird ein leeres Set für die Blacklist angelegt. Diesem Set wird später der Inhalt der Datei übergeben, deren Pfad in *BLACKLIST_FILE* gespeichert wurde. Die Schritte sind voneinander getrennt, um den Effekt der Blacklist durch einen Vergleich der Analyse mit und ohne Blacklist sichtbar zu machen. Auch im Original arbeiten Fischer und Jäschke mit einem zunächst leeren Set, dem bei Bedarf über die Kommandozeile eine Datei übergeben werden kann.  

Die folgenden Variablen speichern verschiedene Regex-Muster, nach denen die übergebenen Dateien durchsucht werden. In der Variable *re_theof* werden verschiedene Regex-Muster gespeichert, mit denen später Vossantos in den NYT-Artikeln identifiziert werden sollen. Fischer und Jäschke haben insgesamt fünf Muster vorbereitet, mit denen die Texte analysiert werden können. Das in der Studie verwendete und standardmäßig voreingestellte Muster ist Nummer 5. Damit wird nach Textstellen gesucht, in denen die Wörter "the" und "of" im Abstand von maximal 5 Wörtern zueinander auftauchen. Da die Variable *re_theof* ein Dictionary ist, kann das gewünschte Muster jeweils durch die zugeordnete Zahl ausgewählt werden.  
Die verschiedenen Muster sind möglicherweise das Ergebnis eines iterativen Prozesses, bei dem alternative Muster getestet und deren Erfolg verglichen wurde. Das fünfte Muster lässt im Vergleich zu den anderen die meisten Wörter und Zusatzzeichen als SOURCE zu. Dadurch, dass auch die anderen Muster Teil der Variable sind, kann bei der Analyse leicht zwischen den verschiedenen Mustern umgeschalten werden. Die Kommentare über jedem einzelnen Muster deuten darauf hin, dass die Muster selbst, die der Methode *re.compile* in Klammern übergeben werden, für Menschen nicht selbsterklärend sind.  
Die beiden Variablen *re_ws* und *control_char_re* enthalten jeweils Muster, mit denen nach Steuerzeichen gesucht wird. Steuerzeichen haben für Menschen keine unmittelbare Bedeutung. Sie geben Maschinen Auskunft über die Art der Darstellung und helfen insofern dabei, Text für Menschen verständlich zu strukturieren (Beispiele sind z. B. Tabs oder Zeilenendezeichen). Da die NYT-Artikel mithilfe von *re_theof* nach festen Zeichenfolgen durchsucht werden, ist es wichtig, nicht relevante Zeichen vorher auszusortieren. Andernfalls könnten Textstellen aussortiert werden, die von Menschen betrachtet als Treffer gezählt werden würden, da sie die Steuerzeichen nicht wahrnehmen.   

An dieser Stelle lohnt sich außerdem die Betrachtung der Variablenbenennung. Dabei fällt auf, dass alle Variablen, denen ein durch die Methode *re.compile* erzeugter Wert zugeordnet wird, das Kürzel "re" enthalten. Dadurch wird eine Ähnlichkeit zwischen den Variablen verdeutlicht und ein direkter Bezug zur Bibliothek *re* hergestellt, den der Computer selbst nicht benötigen würde. Für Menschen ist dadurch nachvollziehbar, dass die Variablen Objekte mit dem Datentyp *Pattern* enthalten (siehe [Dokumentation](https://docs.python.org/3/library/re.html#re-objects)). Mit Ausnahme der letzten Variable steht das Kürzel außerdem an vorderster Stelle im Namen und wird von einer Beschreibung des gesuchten Musters gefolgt. Dieses Vorgehen zur Verknüpfung von Datentyp und Benennung wird noch an anderer Stelle verwendet (siehe Kapitel *Datenaufbereitung*).  
Die Benennung von *control_char_re* ist vermutlich durch den im Kommentar darüber verlinkten Beitrag auf Stack Overflow beeinflusst. Der Kommentar richtet sich mit expliziter Anrede an zukünftige Lesende und legt offen, woher der Ansatz zur Identifizierung von Steuerungszeichen stammt. Damit wird deutlich, dass der Code gezielt aufbereitet wurde, um möglichst verständlich zu sein. Neben der Formulierung von Kommentaren wie diesem deuten auch die umfangreichen Beschreibungen und zusätzlichen Informationen zur [Durchführung der Analyse](https://github.com/weltliteratur/vossanto/blob/master/theof/method.org) darauf hin.

## Datenaufbereitung

Im folgenden Abschnitt werden die XML-Dateien der NYT-Artikel eingelesen und so aufbereitet, dass sie später durchsucht werden können. Um die einzelnen Schritte bei der Aufbereitung nachvollziehen zu können, werden immer wieder exemplarische Ansichten (Inspektionen) des Zustands der Texte gezeigt.

In [3]:
### DEFACTORING INSPECTION
### Überblick über das Verzeichnis mit den NYT-Artikeln

file_amount = len([file for file in os.listdir(NYT_DIRECTORY)])
print(f'Das Korpus der NYT-Artikel enthält für das Jahr 1987 insgesamt {file_amount} Texte.')

Das Korpus der NYT-Artikel enthält für das Jahr 1987 insgesamt 106104 Texte.


Insgesamt kann das Defactoring mit 106.104 Texten durchgeführt werden. Im Vergleich zur Originalstudie, die 1.8 Millionen Artikel der Jahre von 1987-2007 analysiert hat, ist diese Zahl gering. Die Analyse lässt sich dennoch in Teilen reproduzieren. Der Code wurde so geschrieben, dass er mit einer beliebigen Artikelzahl durchgeführt werden kann.

In [4]:
### DEFACTORING FUNCTION
### Einlesen der Dateiliste

def gen_files(path):
    for fname in os.listdir(path):
        fname = path + "/" + fname
        if os.path.isfile(fname):
            yield open(fname, "rt"), fname
            
### DEFACTORING NAMESPACE
files = gen_files(NYT_DIRECTORY)

Mit der Methode *gen_files* werden die XML-Dateien aus dem Dateisystem eingelesen. Sie werden mit dem Methodenaufruf in der Variable *files* gespeichert. Gegenüber dem Originalcode wurde die Methode beim Defactoring deutlich verkürzt. Im Original werden if-else-Abfragen verwendet, um zu prüfen, ob es sich bei der übergebenen Datei um eine einzelne XML-Datei oder ein normales oder komprimiertes Verzeichnis handelt. Weil hier mit relativ großen Datenmengen gearbeitet wird, ist es sinnvoll, die Durchführung sowohl mit einzelnen Dateien als auch mit komprimierten Verzeichnissen zu ermöglichen. So kann das Programm beispielsweise zu Testzwecken mit einer einzelnen Datei gestartet werden, ohne dass eine Anpassung im Code notwendig ist. Im konkreten Fall wird mit einem nicht komprimierten Verzeichnis gearbeitet, weshalb die zusätzlichen Abfragen nicht benötigt werden.  

Der Rückgabewert der Methode wird durch den Ausdruck *yield* geliefert. Damit wird in Python ein *Generatorobjekt* (siehe [Dokumentation](https://www.python.org/dev/peps/pep-0289/)) erzeugt. Ein Generatorobjekt ist ein spezielles Iteratorobjekt und verhält sich auf den ersten Blick ähnlich wie eine Liste. So können die Elemente eines Generators mit einer Schleife iteriert werden. Anders als eine Liste wird mit *yield* jedoch immer nur das gerade benötigte Objekt des Generators zurückgeliefert und der aktuelle Zähler im Iterator festgehalten. Mit dem nächsten Aufruf des Generators wandert dieser interne Zähler weiter und liefert das nächste Objekt zurück. Dadurch muss nicht auf die Erzeugung und das Laden aller Objekte der Liste in den Zwischenspeicher gewartet werden, bevor das gerade benötigte Objekt weiter verwendet werden kann. Die Arbeit mit Generatoren eignet sich daher besonders für große Datensätze und Pipelines, die diese in mehreren Stufen verarbeiten. Der Vorteil in diesem Vorgehen liegt vor allem in einer erhöhten Performance.  
Das Nachvollziehen von Zwischenzuständen solcher Verarbeitungspipelines wird durch Generatoren jedoch erschwert. Da jeder Aufruf des Generators den Wert des internen Zählers verändert, kann während der Ausführung des Programms nicht einfach darauf zugegriffen werden ohne das Programm zu beeinflussen. Mit der Verwendung von Generatoren wird daher eine Entscheidung für Effizienz zulasten der Verfügbarkeit von einsehbaren Zwischenergebnissen getroffen. Aufgrund der Größe der verarbeiteten Daten ist diese Entscheidung durchaus nachvollziehbar. Darüber hinaus entspricht dieses Vorgehen dem in den Digital Humanities beobachteten Fokus auf die Ergebnisse von Forschung (vgl. Andrews 2013 und Ramsay 2011).  
Die Methodenbenennung folgt hierbei dem Schema der Variablenbenennung für reguläre Ausdrücke. Hier wird der Rückgabewert der Methode durch den Methodennamen verdeutlicht. Entsprechend beginnen alle weiteren Methoden des Programms, die ein Generatorobjekt erzeugen, mit dem Kürzel "gen". Anders als bei statisch typisierten Programmiersprachen wie Java muss der Rückgabewert einer Methode in Python nicht bei deren Definition festgelegt werden. Daher dient diese Information nur der Verständlichkeit für Menschen. Durch die Benennung wird außerdem ein Zusammenhang zwischen den wichtigsten Methoden der Textverarbeitung in dem Programm hergestellt.

Für ein bessers Verständnis der folgenden Schritte, in denen die eingelesenen XML-Dateien verarbeitet werden, soll zunächst ein Überblick über deren Aufbau gegeben werden.

In [5]:
### DEFACTORING INSPECTION

### Überblick über Dateipfad zur XML-Datei eines Artikels
filepath = NYT_DIRECTORY + '/' + os.listdir(NYT_DIRECTORY)[0]
print(f'Dateipfad zur ersten Datei im Verzeichnis der NYT-Artikel: {filepath}')

Dateipfad zur ersten Datei im Verzeichnis der NYT-Artikel: nyt_corpus_1987/0027572.xml


In [6]:
### DEFACTORING INSPECTION
### Beispielausgabe einer XML-Datei

with open(filepath, 'r') as example_file:
    print(example_file.read())

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE nitf SYSTEM "http://www.nitf.org/IPTC/NITF/3.3/specification/dtd/nitf-3-3.dtd">
<nitf change.date="June 10, 2005" change.time="19:30" version="-//IPTC//DTD NITF 3.3//EN">
  <head>
    <title>HERO OF THE LIFE CYCLE</title>
    <meta content="5" name="publication_day_of_month"/>
    <meta content="4" name="publication_month"/>
    <meta content="1987" name="publication_year"/>
    <meta content="Sunday" name="publication_day_of_week"/>
    <meta content="Book Review Desk" name="dsk"/>
    <meta content="36" name="print_page_number"/>
    <meta content="7" name="print_section"/>
    <meta content="2" name="print_column"/>
    <meta content="Arts; Books" name="online_sections"/>
    <docdata>
      <doc-id id-string="27572"/>
      <doc.copyright holder="The New York Times" year="1987"/>
      <series series.name="MIND/BODY/HEALTH"/>
      <identified-content>
        <classifier class="indexing_service" type="descriptor">BOOK REVIEWS</class

Die Ausgabe einer Beispieldatei des NYT-Korpus zeigt, dass diese zusätzlich zum Text des Artikels eine Reihe Metaangaben enthält. Der Artikelinhalt befindet sich im XML-Tag *`<block class="full_text">`*. In den folgenden Schritten der Auswertung werden diese Auszeichnungen durchsucht und in Textform gespeichert. Fischer und Jäschke speichern zunächst alle Meta-Angaben und den Artikelinhalt in Form von eigenen Variablen.

In [7]:
def xml2str(f):
    tree = ET.parse(f)
    root = tree.getroot()

    # path to the heading
    #
    # ./body/body.head/hedline vs. ./body/body.head/
    # ./body/body.head/ and ./body/body.head/hedline include only the
    # heading itself, not the first introductory sentence which we get
    # with ./body/body.head
    heading = root.findall("./body/body.head")
    head = ""
    for block in heading:
        head += ET.tostring(block, encoding="utf-8", method="text").decode("utf-8")

    # path to the main text block
    content = root.findall("./body/body.content/")
    body = ""
    for block in content:
        if block.attrib["class"] == "full_text":
            # strip XML
            body += "\n" + ET.tostring(block, encoding="utf-8", method="text").decode("utf-8")
    return head + "\n\n" + body

def gen_text(files):
    for f, fname in files:
        yield fname, xml2str(f)

### DEFACTORING NAMESPACE
### Erzeugen einer Liste aus Texten
texts = gen_text(files)

### Aufteilung der Texte in Sätze
def gen_sentences(texts):
    for fname, text in texts:
        # split at line breaks, since the XML preserved paragraphs
        for line in text.splitlines():
            for sentence in sent_tokenize(line):
                yield fname, sentence

### DEFACTORING NAMESPACE
sents = gen_sentences(texts)

An dieser Stelle geht die Verarbeitungspipeline der NYT-Artikel mit Generatorobjekten weiter. Die Methode *xml2str* ist eine Hilfsmethode, die von *gen_text* verwendet wird. Sie verwendet das zu Beginn importierte XML-Modul, mit dem die XML-Dateien geparst werden können. Dabei werden zunächst sämtliche Metadaten zum Artikel gespeichert. Anschließend wird der Artikelinhalt gesucht und hinzugefügt. Die Kommentare in der Methode zeigen die Pfade zu den Metaangaben (./body/body.head) bzw. dem Artikel (./body/body.content) an. Die Methode *gen_files* verwendet *xml2str* und speichert den Dateiinhalt jeweils zusammen mit dem dazugehörigen Dateinamen.  
Anschließend werden die Texte mit der Methode *gen_sentences* in Sätze eingeteilt. Auch hier wird für jeden Satz der dazugehörige Dateiname gespeichert, um die Zuordnung zum entsprechenden Artikel zu gewährleisten. Für die Aufteilung der Texte in Sätze wird die NLTK-Methode *sent_tokenize* verwendet. Mit diesem Schritt sind die Artikel in sehr kleine Einheiten aufgeteilt und aufbereitet, damit sie anschließend nach Vossantos durchsucht werden können.

## Auswertung

Die Auswertung erfolgt in mehreren Schritten, bei denen die Regeln für mögliche Vossantos immer weiter verschärft werden. Zuerst werden die Texte nach dem Muster *"the SOURCE of"* durchsucht. Anschließend wird die Trefferliste mit der Personenliste aus dem Datensatz von Wikidata abgeglichen. Als letztes wird die Blacklist hinzugezogen. 

### Schritt 1: Regex-Muster

In [8]:
def gen_regex(texts, regex, groupid=0):
    for fname, text in texts:
        for m in regex.finditer(text):
            # also return the start position of the matches
            yield fname, m.group(groupid), m.start(groupid), text
            
def gen_rm_ctrl(texts):
    for cols in texts:
        #yield [re_ws.sub(' ', col) for col in cols]
        res = []
        for col in cols:
            if isinstance(col, int):
                res.append(col)
            else:
                res.append(re_ws.sub(' ', col))
        yield res

### DEFACTORING NAMESPACE        
### Anwendung des Regex-Musters
match = gen_regex(sents, re_theof[5])
match = gen_rm_ctrl(match)

Im ersten Schritt der Auswertung wird weiter mit Generatorobjekten gearbeitet. Die Methode *gen_regex* durchsucht die zuvor eingeteilten Sätze der NYT-Artikel (die in der Variable *sents* gespeichert wurden) nach dem übergebenen Regex-Muster, welches mit *re_theof[5]* standardmäßig voreingestellt ist. Durch Anpassung der Zahl kann hier auf die verschiedenen, in *re_theof* gespeicherten, Muster zurückgegriffen werden. Standardmäßig werden alle Sätze, die dem Muster "the SOURCE of" entsprechen, in der Trefferliste *match* gespeichert. Anschließend werden die Treffer um die in *re_ws* gespeicherten Steuerzeuchen wie Tabs und Zeilenendezeichen mit der Methode *gen_rm_ctrl* aus der Trefferliste entfernt.  
Bei der Methode fällt auf, dass darin die zweite Zeile Code auskommentiert ist. In der einen Zeile werden bis auf einen Unterschied die gleichen Schritte durchgeführt wie in den darauf folgenden Zeilen der Methode. Allerdings fehlt dabei die *if-else* Kontrollstruktur. Diese prüft durch die Methode *isinstance*, ob die Werte der Spalten Zahlenwerte enthalten. Nur bei Werten, die keine Zahlen sind, wird die Bereinigung um Leerzeichen durchgeführt. Da diese Kontrolle in der auskommentierten Zeile fehlt, würde dort auch bei Zahlen versucht, Leerzeichen zu entfernen. Diese Operation ist jedoch semantisch falsch, daher würde das Programm einen Fehler werfen und abbrechen.  

Anders als beim sonstigen Vorgehen werden hier die Ergebnisse zwei verschiedener Methoden in der selben Variable gespeichert. Sowohl die Trefferliste der Vossanto-Muster als auch deren Bereinigung um Steuerzeichen werden in *match* gespeichert, wodurch die unbereinigte Trefferliste überschrieben wird. Sie steht damit nicht mehr für weitere Berechnungen zur Verfügung. Im Gegensatz dazu wurden bisher alle Zwischenergebnisse der Textverarbeitung in eigenen Variablen gespeichert. Diese Abweichung lässt sich möglicherweise dadurch erklären, dass der Unterschied in der Trefferliste mit und ohne Steuerzeichen für Menschen keine Bedeutung spielt – er ist nur für die Behandlung der Texte durch den Computer relevant.

In [9]:
###GROßER VERARBEITUNGSAUFWAND###
### DEFACTORING FUNCTION

def print_matches(matches, output_file, sep='\t'):
    for fname, match, index, text in matches:
        print(fname, match, text, file=output_file, sep=sep)


with open('theof_1987.tsv', 'w') as output:
    print_matches(match, output, '\t')

In dieser Zelle endet die Verarbeitungspipeline mit Generatoren. Die Methode *print_matches* gibt die Trefferliste möglicher Vossantos als Tabelle aus. Die Methode wurde gegenüber dem Original soweit angepasst, dass die Ausgabedatei schon in der Methode übergeben wird. Im Original wird die Ausgabe der Ergebnisse über die Kommandozeile geregelt, weshalb der Eingabeparameter *output_file* dort nicht existiert.  
Das print-Statement der Methode gibt Auskunft über den Aufbau der Tabelle: Für jeden Treffer werden darin der Dateiname, das identifizierte Muster, sowie der vollständige Satz, in dem der Treffer gefunden wurde, ausgegeben.  
Mit der for-Schleife werden alle Objekte des Generators abgefragt und in die Tabelle geschrieben. Wegen der Größe des Datensatzes ist diese Operation relativ aufwendig und kann, je nach System auf dem sie ausgeführt wird, lange dauern. Daher muss sie nicht zwingend ausgeführt werden. Der folgende Code arbeitet mit einer vorbereiteten Datei weiter (die durch die Endung "_prep" im Dateinamen gekennzeichnet ist).

In [10]:
### DEFACTORING INSPECTION
### Übersicht über die häufigsten Treffer
!wc -l theof_1987_prep.tsv

641432 theof_1987_prep.tsv


Zur Veranschaulichung von Zwischenschritten setzen Fischer und Jäschke auf Shellbefehle. Diese Art von Befehlen wird auch in diesem Notebook verwendet um Zwischenzustände der Analyse genauer zu betrachten. Hier werden durch das Programm *wc* die Zeilen der erzeugten Datei gezählt. Dadurch lässt sich überprüfen, wie viele Treffer durch den ersten Ansatz zur Identifizierung der Vossantos gefunden wurden.

Insgesamt wurden 641.432 potenzielle Treffer für Vossantos gefunden. Die folgende Zelle gibt einen Überblick über deren Inhalt.

In [11]:
### DEFACTORING INSPECTION
### Überblick über die Datei

!awk -F'\t' '{print $2}' theof_1987_prep.tsv| sort | uniq -c | sort -nr | head

   7102 the end of
   6606 the University of
   4287 the number of
   3206 the sale of
   3172 the rest of
   2370 the use of
   2172 the kind of
   2135 the son of
   1943 the age of
   1883 the president of
sort: Schreiben fehlgeschlagen: Standardausgabe: Datenübergabe unterbrochen (broken pipe)
sort: Schreibfehler


Die Ausgabe zeigt die 10 häufigsten Treffer möglicher Vossanto im NYT-Datensatz und deren Häufigkeit. Diese werden in erster Linie durch das Programm *awk* (siehe [Dokumentation](http://www.gnu.org/software/gawk/manual/html_node/index.html)) ermittelt. Awk liest die Tabelle der Vossantotreffer ein und gibt durch *{print $2}* jeweils die Werte der zweiten Spalte zurück. Darin sind die potenziellen Vossantos gespeichert. Durch *sort* werden diese alphabetisch sortiert. Mit *uniq -c* wird gezählt, wie häufig einzelne Treffer vorkommen. Diese werden anschließend mit *sort -nr* nochmals nach der Häufigkeit des Aufkommens sortiert. Die Fehlermeldung am Ende der Tabelle wird durch *sort* verursacht, da das Programm alle Ergebnisse ausgeben soll. Mit dem Kommando *head* wird die Ausgabe jedoch nach den ersten 10 Treffern beendet, weshalb *sort* seine Aufgabe nicht vollständig erfüllen kann.  

Die Ausgabe zeigt, dass die Suche nach Vossantos allein auf Basis vorgegebener Zeichen wenig bei der Identifizierung von Vossantos hilft. Keiner der 10 häufigsten Treffer enthält den Namen einer Person. Um diese *false positives* auszusortieren, wird im nächsten Schritt die Personenliste von Wikidata hinzugezogen.

### Schritt 2: Regex-Muster und Wikidata

#### Wikidata-Download

Zunächst werden mithilfe des [Wikidata Toolkit](https://www.mediawiki.org/wiki/Wikidata_Toolkit) sämtliche Datenbankeinträge von Wikidata heruntergeladen. Das Wikidata Toolkit ist eine Java-Bibliothek. Sie stellt Klassen und Methoden zur Verarbeitung von Daten aus Wikidata bereit. Mit diesen kann der Datendump nach dem Download nach allen Einträgen durchsucht werden, die zu einer Person gehören. Diese Art der Abfrage ist möglich, da Wikidata strukturierte Daten in Form von *Linked Open Data* enthält. Alle Einträge (*Items*) werden durch bestimmte Aussagen beschrieben. Diese Aussagen bestehen aus Attributen (*properties*) und Werten (*values*). Konkret wird bei der Abfrage nach allen Einträgen gefragt, die das Attribut *P31* (entspricht *instanceOf*) mit dem Wert *Q5* (entspricht *human*) haben.  
Fischer und Jäschke haben zwar den Code zur Erzeugung der Personenliste veröffentlicht. Dieser Schritt der Auswertung lässt sich dennoch nicht reproduzieren, da der originale Datensatz von Wikidata nicht rekonstruiert werden kann. Wikidata wird ständig bearbeitet und vollständige Zustände der Datenbank werden nur für etwas mehr als einen Monat gespeichert. Zum Vergleich: Die Personenliste von Fischer und Jäschke wurde vor dem 19.07.2017 erzeugt (siehe [commit](https://github.com/weltliteratur/vossanto/commit/3a1e3f179c35807b0f2474a729ae8fae0140ec37) im Github Repositorium). Zu diesem Zeitpunkt enthielt Wikidata etwa 29 Millionen Einträge (siehe [Wikidata-Statistik](https://tools.wmflabs.org/wikidata-todo/stats.php)). Für Dezember 2019 zeigt die Statistik etwa 70 Millionen Datenbankeinträge. Da die Einträge keine Information über ihr Erstelldatum enthalten, ist ohne einen Vergleich der beiden Datensätze nicht nachvollziehbar, welche Einträge im Vergleich zu Juli 2017 hinzugekommen sind.  

Auch ohne Ausführung bietet der Code durchaus einen Mehrwert für das Verständnis des Umgangs mit der Schnittstelle von Wikidata. Darüber hinaus besteht eine Abhängigkeit der hier verwendeten Personenliste vom Code. Da er für die Reproduzierung der Studienergebnisse jedoch nicht ausgeführt werden kann, wird er in diesem Notebook nicht näher berücksichtigt. Hier muss zwischen einer möglichst umfassender Betrachtung und einem Fokus auf für den Kontext relevanten Code abgewogen werden. Schmidt schreibt dazu: "The underlying complexity of computers makes some degree of ignorance unavoidable." (2016). Der Code hat auf die Ausführung des Programms im Rahmen des Defactorings keine direkte Auswirkung und seine Ausführung lässt sich nicht unmittelbar nachvollziehen. Aus diesem Grund wird an dieser Stelle mit der von Fischer und Jäschke bereitgestellten Personenliste weiter gearbeitet.

In [12]:
### DEFACTORING INSPECTION
### Überblick über die Wikidata-Personenliste
!wc -l wikidata_humans_prep.tsv

2801931 wikidata_humans_prep.tsv


Die Inspektion zeigt, dass die Personenliste insgesamt 2.801.931 Einträge enthält. Diese Zahl entspricht allen Einträgen von Personen in Wikidata zum Zeitpunkt des Downloads von Fischer und Jäschke, die ein Label (also einen Namen) haben.

In [13]:
### DEFACTORING INSPECTION
### Überblick über Inhalt der Liste
!awk -F'\t' '{print $1 $2 $3 $4 $5}' wikidata_humans_prep.tsv | head

"Q23""George Washington""Father of the United States""Washington""President Washington"
"Q42""Douglas Adams""Douglas Noël Adams""Douglas Noel Adams""Douglas N. Adams"
"Q76""Barack Obama""Barack Hussein Obama II""Barack Obama II""Barack Hussein Obama"
"Q80""Tim Berners-Lee""TimBL""Sir Tim Berners-Lee""Timothy John Berners-Lee"
"Q91""Abraham Lincoln""Abe Lincoln""Lincoln""Honest Abe"
"Q157""François Hollande""François Gérard Georges Nicolas Hollande""Francois Hollande""Hollande"
"Q181""Jimmy Wales""Jimbo Wales""Jimmy Donal Wales""Jimbo"
"Q185""Larry Sanger""Lawrence Mark Sanger""Lawrence Mark ""Larry"" Sanger"
"Q186""Ken Jennings""Kenneth Wayne ""Ken"" Jennings III"
"Q192""David Cameron""David William Donald Cameron"
awk: write failure (Broken pipe)
awk: close failed on file /dev/stdout (Broken pipe)


Der Überblick verdeutlicht den Aufbau der Tabelle: Jede Person hat eine eindeutige ID, die jeweils mit "Q" beginnt und von einer Zahl gefolgt wird. Durch diese ID sind die Einträge in Wikidata eindeutig identifizierbar. Auf die ID folgt der Personenname. Anschließend werden Aliase oder Synonyme für die Person aufgelistet. Diese Zuordnung verschiedener Benennungen ist für die Vossanto-Analyse besonders hilfreich. So können auch verschiedene Bezeichnungen der selben Person als solche erkannt und einander zugeordnet werden.

Die Fehlermeldung zum Ende des Programms wird auch hier wieder durch das Kommando *head* versursacht, welches *awk* daran hindert, die Ausgabe aller Zeilen durchzuführen.

In [14]:
### DEFACTORING NAMESPACE

def remove_control_chars(s):
    return control_char_re.sub('', s)

def clean(s):
    s = s[1:-1]
    # unquote
    s = re_quotes.sub("\"", s)
    # remove non-printable characters
    # TODO: does not work properly for U+200E (which is changed to E2)
    # check with "Q29841121"     "Alexander Howe<U+200E>"
    s = remove_control_chars(s)
    return s

def get_items(fname, sep='\t'):
    items = dict()
    synonyms = dict()
    # format: "Q863081"       "Billy ""The Kid"" Emerson"
    with open(fname, "rt", encoding="utf-8") as f:
        for line in f:
            # extract parts
            itemId, itemLabels = line.strip().split(sep, 1)
            itemLabels = itemLabels.split(sep)
            itemLabel = itemLabels.pop(0)
            # remove surrounding quotes
            itemId = clean(itemId)
            itemLabel = clean(itemLabel)
            # ignore duplicates
            if itemLabel not in items:
                items[itemLabel] = itemId
            elif int(itemId[1:]) < int(items[itemLabel][1:]):
                # ensure that we always use the item with the lowest id
                items[itemLabel] = itemId
            # handle synonyms
            for syn in itemLabels:
                syn = clean(syn)
                # conditions as before (but merged with or)
                if syn not in synonyms or int(itemId[1:]) < int(items[itemLabel][1:]):
                    # we store the itemLabel such that we can easily
                    # print it and get the corresponding itemId via
                    # items
                    synonyms[syn] = itemLabel
    return items, synonyms

### DEFACTORING NAMESPACE
# Read list of persons from Wikidata
items, synonyms = get_items(WIKIDATA_HUMANS)

Der Code in dieser Zelle liest die Wikidata-Personenliste ein und überträgt die Einträge darin in ein Dictionary, sodass jeder ID der zugehörige Personenname zugeordnet wird. Außerdem wird ein weiteres Dictionary für die Synonyme angelegt. Dadurch können alle Namen und deren Synonyme eindeutig über die ID identifiziert werden.  
Die beiden Methoden *remove_control_chars* und *clean* sind Hilfsmethoden für *get_items*. Sie entfernen Steuer- und Leerzeichen aus dem Datensatz, indem die zu Beginn des Programms definierten regulären Ausdrücke verwendet werden. Der Funktionsname *clean* deutet an, dass der Datensatz von den entfernten Zeichen gesäubert wird, weil sie für Menschen keine Bedeutung haben oder sogar stören, weil sie den Lesefluss unterbrechen. Für den Vergleich der Personennamen aus der Datenbank von Wikidata mit Textstellen der NYT-Artikel ist eine identische Zeichenfolge dagegen sehr wichtig.  
Die Kommentare in der Methode  weisen darauf hin, dass diese Ausdrücke noch nicht für alle Steuerzeichen in der Personenliste funktionieren, weil sie nicht vollständig entfernt werden. Der nicht abgeschlossene Zustand des Programms spricht dafür, dass Fischer und Jäschke das Problem für vernachlässigbar halten. Es bleibt jedoch unklar, welche konkrete Auswirkung die fehlerhafte Bereinigung auf das Ergebnis der Trefferliste hat.    

Die Methode *get_items* liest die Personenliste zeilenweise ein und erzeugt daraus zwei Dictionaries. Das Dictionary *items* speichert den Personennamen als Schlüssel und die dazugehörige ID als Wert. Diese Aufteilung erscheint auf den ersten Blick widersprüchlich, da sie die Bedeutung von ID und Entität umdreht. Die Erklärung dafür liefern Fischer und Jäschke im letzten Kommentar der Methode selbst ("# we store the itemLabel such that we can easily print it and get the corresponding itemId via items"). Sie ergibt sich aus der Kombination mit dem zweiten Dictionary, *synonyms*. Darin wird das Synonym für die Person als Schlüssel und der Personenname als Wert gespeichert. Da die Einträge von Dictionaries in Python über den Ausdruck *dictionary\[schlüssel\]* ausgegeben werden, kann durch die Zuordnung in *items* und *synonyms* problemlos eine Verknüpfung zwischen Synonymen und den zugehörigen IDs hergestellt werden, obwohl diese in verschiedenen Dictionaries gespeichert werden. Ein Beispiel dafür liefert die folgende Inspektionszelle.  
Dass hier mit Daten von Wikidata gearbeitet wird, wird auch durch die Variablennamen deutlich. Die Bezeichnungen *itemId* und *itemLabel* sind Begriffe aus dem Wikidata Toolkit. Außerdem ist die Schreibweise im camelCase eine Konvention in Java und für Python unüblich.

In [15]:
### DEFACTORING INSPECTION

synonym = 'Maggie Thatcher'

print(f'"{synonym}" ist ein Synonym für "{synonyms[synonym]}". Sie hat die ID {items[synonyms[synonym]]}.')

"Maggie Thatcher" ist ein Synonym für "Margaret Thatcher". Sie hat die ID Q7416.


Durch die zwei Dictionaries können alle Namen und deren Synonyme eindeutig über die ID identifiziert werden. Diese Personenliste wird jetzt herangezogen, um die erzeugte Liste potenzieller Vossanto-Treffer damit abzugleichen.

#### Abgleich der Datensätze

In [16]:
### DEFACTORING FUNCTION

def compare_lists(input_file, output_file):
    with open(input_file) as f:
        with open(output_file, 'w') as output_wd:
            for line in f:
                article, phrase, sentence = line.strip().split('\t', 2)
                # the John Doe of -> strip "the" and "of"
                item = phrase[4:-3]
                # check if exists
                if item in items and item not in blacklist:
                    print(article, items[item], phrase, item, item, sentence, file=output_wd, sep='\t')
                elif item in synonyms and item not in blacklist:
                    print(article, items[synonyms[item]], phrase, item, synonyms[item], sentence, file=output_wd, sep='\t')
                else:
                    # check if the phrase itself contains "the"
                    for match in re_the.findall(item):
                        if match in items and match not in blacklist:
                            print(article, items[match], phrase, match, match, sentence, file=output_wd, sep='\t')
                        elif match in synonyms and match not in blacklist:
                            print(article, items[synonyms[match]], phrase, match, synonyms[match], sentence, file=output_wd, sep='\t')
                            
compare_lists('theof_1987_prep.tsv', 'theof_1987_wd.tsv')    

Die Methode *compare_lists* existiert im Originalcode nicht. Stattdessen wird der darin verwendete Code in der Main-Methode des Skripts *check_wikidata.py* ausgeführt. In diesem Notebook wird der Code zweimal ausgeführt, um den Effekt der Blacklist auf das Ergebnis zu verdeutlichen. Für die Wiederverwendbarkeit wurde er daher in einer Methode gespeichert. Außerdem wird das Ergebnis des Codes im Original durch die Kommandozeile in eine Datei umgeleitet – hier wird die Ausgabedatei wie bei *print_matches* im print-Statement selbst festgelegt.  

An dieser Stelle des Codes erfolgt der Vergleich von Vossanto-Treffern durch das Regex-Muster mit der Personenliste von Wikidata. Innerhalb der Methode erfolgt bereits der Abgleich mit der Blacklist (vgl. Code nach "# check if exists"). Sie ist zu diesem Zeitpunkt noch leer und hat dadurch keinen Einfluss auf das Ergebnis.  
Die potenziellen Treffer werden um die Begriffe "the" und "of" gekürzt, damit nur der Name übrig bleibt. Anschließend wird überprüft, ob dieser Name entweder im Dictionary *items* oder *synonyms* vorkommt. Gleichzeitig wird überprüft, ob der Name nicht auf der Blacklist steht. Wenn beide Überprüfungen positiv verlaufen, wird der Treffer in die Ausgabedatei geschrieben.  
Damit sind jedoch nicht alle Fälle abgedeckt. Die erste Abfrage funktioniert nur für Zeichenfolgen, bei denen innerhalb von "the" und "of" nur ein Personenname steht. Da das standardmäßig verwendete Regex-Muster jedoch bis zu fünf Wörter zwischen "the" und "of" zulässt, geht diese Einschränkung teilweise zu weit. Ein Beispiel dafür ist die Phrase "the director and the writer of". Auch mit der Kürzung verbleibt "director and the writer". Selbst wenn "writer" ein Personenname wäre, würde er durch die vorangehenden Begriffe nicht in der Personenliste von Wikidata gefunden werden. Daher wenden Fischer und Jäschke eine weitere Überprüfung an (vgl. Code nach '# check if the phrase itself contains "the"'): Die Phrase selbst wird nochmals nach "the" durchsucht. Für den darauf folgenden Begriff wird anschließend wieder geprüft, ob dieser in der Wikidata-Personenliste enthalten ist und nicht auf der Blacklist steht.  
Hier wird die Schwierigkeit beim Umgang mit natürlicher Sprache sehr deutlich. Einfache Muster können die vielen Möglichkeiten bei der Verwendung natürlicher Sprache häufig nicht vollständig abdecken. Gleichzeitig haben sehr spezifische Muster den Nachteil, dass ggf. zu stark aussortiert wird. In solchen Fällen muss eine Abwägung zwischen Genauigkeit und Treffermenge getroffen werden.  

Mit der nächsten Inspektion soll ein Überblick über den Effekt der Personenliste auf die identifizierten Treffer gegeben werden.

In [17]:
### DEFACTORING INSPECTION
!wc -l theof_1987_wd.tsv

5236 theof_1987_wd.tsv


Insgesamt enthält die Trefferliste noch 5.236 Einträge. Im Vergleich zu den 2.8 Millionen Einträgen aus dem Durchlauf ohne die Personenliste sind das deutlich weniger. Die nächste Zelle wird einen Eindruck geben, ob falsche Treffer damit in bedeutsamer Weise aussortiert werden konnten.

In [18]:
### DEFACTORING INSPECTION
### Überblick über die häufigsten Treffer
!awk -F'\t' '{print $3}' theof_1987_wd.tsv| sort | uniq -c | sort -nr | head

    746 the House of
    407 the Bureau of
    374 the image of
    312 the Court of
    310 the wife of
    220 the Church of
    211 the Hall of
    171 the Mayor of
    120 the Bill of
    108 the star of
sort: Schreiben fehlgeschlagen: Standardausgabe: Datenübergabe unterbrochen (broken pipe)
sort: Schreibfehler


Nach wie vor enthält die Liste der zehn häufigsten Treffer keine richtigen Vossantos. Das liegt u. a. Problem darin, dass die Wikidataliste Personen mit Namen wie "Hall" oder "Church" enthält. Daher werden diese Begriffe nicht aussortiert. Fischer und Jäschke setzen daher zusätzlich auf eine Blacklist.

### Schritt 3: Regex-Muster, Wikidata und Blacklist

Die Erzeugung der Blacklist ist der einzige Schritt in der Durchführung der Vossanto-Studie, der nicht durch Code abgebildet ist. Daher ist ihr Zustandekommen nicht reproduzierbar. Fischer und Jäschke geben an, dass sie die Blacklist auf Basis der Treffer angelegt haben, die bei der Suche des Regex-Musters nach dem Abgleich mit der Personenliste gefunden wurden. Davon wurden jeweils die Begriffe auf die Blacklist gesetzt, bei denen die Wahrscheinlichkeit hoch ist, dass sie beim Vorkommen in den Artikeln keine Person beschreiben.  
Durch dieses Vorgehen ist die Blacklist sehr stark von den Ausgangsdatensätzen sowie der Einschätzung der Studienautoren abhängig. Ersteres bedeutet für das Defactoring, dass es nicht möglich ist, mit einer neueren Version eines Datendumps von Wikidata zu arbeiten. Dieser würde viele Begriffe enthalten, die im Datensatz der Originalstudie noch nicht vorhanden waren – und damit auch nicht in die Blacklist übernommen wurden. Ein Beispiel dafür ist Andrew John Henry Way (mit der ID Q17641254), der vermutlich erst nach dem 19.07.2017 zur Datenbank von Wikidata hinzugefügt wurde. Zumindest steht dieser Name nicht in der Personenliste von Fischer und Jäschke. Das hat zur Folge, dass die Durchführung von *compare_lists* mit einem Datendump vom 02.12.2019 jedes Vorkommen der Phrase "the way of" als Treffer wertet. Daher wird neben der vorgegebenen Personenliste auch mit der von Fischer und Jäschke veröffentlichten Blacklist gearbeitet.

#### Erzeugung der Blacklist

In [19]:
### DEFACTORING INSPECTION
!wc -l blacklist_prep.tsv

1289 blacklist_prep.tsv


Die Blacklist enthält 1.289 Einträge. Da sie unter Berücksichtigung NYT-Artikel aller Jahrgänge von 1987-2007 angelegt wurde, enthält sie viele Begriffe, die in keinem der Texte von 1987 vorkommen. Das ist an dieser Stelle unproblematisch, da die Blacklist im Code von *compare_lists* nur nach Begriffen durchsucht wird, die vorher in einem Text gefunden wurden.

In [20]:
### DEFACTORING INSPECTION
### Überblick über die Blacklist

!awk -F'\t' '{print $1}' blacklist_prep.tsv| head -15

House
Church
Hall
Bill
Freedom
Governor
Sultan
Duke
King
Chancellor
Prince
Princess
Life
Spirit
Kingdom
awk: write failure (Broken pipe)


Die Inspektionszelle zeigt die ersten 15 Einträge der Blacklist. Fischer und Jäschke weisen selbst darauf hin, dass davon bis auf 62 Ausnahmen alle Begriffe aus nur einem Wort bestehen. Zum Beleg dieser Behauptung durchsuchen sie die Tabelle mit dem Programm *grep* nach Begriffen, die Leerzeichen enthalten:

In [21]:
### DEFACTORING INSPECTION
!grep " " blacklist_prep.tsv | wc -l

62


Die ersten Begriffe auf der Blacklist zeigen, dass eine Verwendung im Rahmen einer Vossanto eher unwahrscheinlich ist. Dennoch ist es denkbar, dass durch einen Begriff auf der Blacklist eine richtige Vossanto aussortiert wird. Durch die Blacklist werden falsche Treffer wie "the Prince of Wales" aussortiert. Gleichzeitig würden auch solche Fälle nicht berücksichtigt, in denen Bezug auf den Musiker genommen wird. Für die Unterscheidung der Verwendung von "Prince" als Adelstitel oder als Musikername ist Kontextwissen notwendig, welches durch maschinelle Verfahren nicht einfach erzeugt werden kann. Fischer und Jäschke treffen hier eine bewusste Abwägung zwischen der Berühmtheit einer Person bzw. der Wahrscheinlichkeit von deren Verwendung in einer Vossanto und der Häufigkeit falscher Treffer.  
An dieser Stelle wäre eine genauere Auseinandersetzung mit dem Konzept der *Berühmtheit* spannend. Da Fischer und Jäschke keine Definition dafür bereitstellen und auch der Workflow zur Erzeugung der Blacklist nicht klar definiert ist, bleibt nur eine vage Vermutung. Hier zeigt sich die Schwierigkeit bei der Übertragung unscharfer gesellschaftlicher Konzepte in maschinell verarbeitbare Strukturen.

In [22]:
def get_blacklist(fname, sep='\t'):
    items = set()
    # format: Lone Ranger\t7
    with open(fname, "r", encoding="utf-8") as f:
        for line in f:
            # extract parts
            item, count = line.strip().split(sep)
            items.add(item)
    return items

### DEFACTORING NAMESPACE
blacklist = get_blacklist(BLACKLIST_FILE)

Mit der Methode *get_blacklist* wird die Blacklistdatei eingelesen und in einem Set gespeichert. Ein Set ist hier ein geeigneter Datentyp, da dadurch sichergestellt wird, dass Begriffe nicht mehrmals in der Liste auftauchen. Damit enthält die bereits verwendete Variable *blacklist* nun sämtliche Begriffe auf der Blacklist.

#### Abgleich der Datensätze

In [23]:
compare_lists('theof_1987_prep.tsv', 'theof_1987_wd_bl.tsv')

Der Aufruf unterscheidet nur marginal vom vorherigen Aufruf der Methode *compare_lists*. Der Unterschied liegt nur in der dabei verwendeten Variable *blacklist* sowie der Ausgabedatei, welche zusätzlich das Kürzel "_bl" im Namen trägt. Die Auswirkungen der Blacklist auf das Ergebnis der Trefferliste wird in der folgenden Zelle untersucht.

In [24]:
### DEFACTORING INSPECTION
!wc -l theof_1987_wd_bl.tsv

131 theof_1987_wd_bl.tsv


Mit der Kombination der drei Schritte zur Identifizierung der Vossantos werden nurnoch 131 potenzielle Treffer gefunden. Die folgende Inspektion zeigt die häufigsten davon.

In [25]:
### DEFACTORING INSPECTION
### Überblick über die häufigsten Treffer
!awk -F'\t' '{print $4}' theof_1987_wd_bl.tsv| sort | uniq -c | sort -nr | head

      4 Horatio Alger
      4 Frank Sinatra
      3 Woody Allen
      3 Madonna
      2 Tom Seaver
      2 Pete Rose
      2 Joan Baez
      2 Jackie Robinson
      2 Groucho Marx
      2 Goebbels


Mithilfe der Blacklist zeigen die zehn häufigsten Treffer richtige Vossantos. Damit ist die Kombination aus Regex-Muster, Wikidataeinträgen und manueller Blacklist der beste der bisher geprüften Ansätze zur Identifizierung von Vossantos.

## Darstellung der Ergebnisse

Da sich die in *"The Michael Jordan of greatness". Extracting Vossian antonomasia from two decades of The New York Times, 1987-2007* dargestellten Ergebnisse auf die Analyse des Gesamtkorpus mit allen Artikeln beziehen, können diese mit dem Teilkorpus nicht nachvollzogen werden. Fischer und Jäschke konzentrieren sich bei der Besprechung der Ergebnisse auf folgende Kategorien:

1. die häufigsten Vossanto SOURCEs
2. die häufigsten Vossanto MODIFIER
3. Rubriken der NYT, in denen Vossantos am häufigsten vorkommen
4. Autorinnen und Autoren, die Vossantos besonders häufig verwenden

Wegen der ausführlichen Dokumentation der Methode und weiterer Statistiken lassen sich zumindest die Ergebnisse der ersten Kategorie vergleichen. Die in der letzten Inspektionszelle gezeigte Übersicht über die 10 häufigsten Vossanto-Treffer entspricht den von Fischer und Jäschke veröffentlichten Zahlen für die Artikel aus dem Jahr 1987. 
Durch den veröffentlichten Code wäre es zwar möglich, auch für die Kategorien 2-4 Statistiken zu berechnen. Dafür müsste die Tabelle mit den Treffern zunächst in eine Datei des Auszeichnungsformats *.org* konvertiert werden. Anschließend haben Fischer und Jäschke die MODIFIER und andere Teile der Vossantos manuell gekennzeichnet. Auf Basis dieser Kennzeichnung kann die Datei anschließend analysiert und grafisch aufbereitet werden. Da für das Jahr 1987 keine Vergleichswerte vorliegen, ist eine grafische Darstellung der Ergebnisse an dieser Stelle nicht zielführend um die Studienergebnisse zu reproduzieren. Dieses Problem verdeutlicht einmal mehr die Abhängigkeit von Code und Daten.